# **LangChain con Python y Gemini AI**

Proyecto desarrollado en Python, implementando LangChain con Gemini, para realizar preguntas con contexto usando como ejemplos una API, un documento PDF, un audio y una base de datos SQL Server.

In [7]:
!pip install -q langchain openai google-generativeai requests langchain-google-genai faiss-cpu pypdf langchain-community openai-whisper sqlalchemy sqlalchemy-pytds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.3 MB/s eta 0:00:00


In [18]:
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import whisper
from langchain.sql_database import SQLDatabase
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import urllib
from sqlalchemy import create_engine

# **LangChain usando una API**

In [77]:
OPENWEATHER_API_KEY = "XXXXXXXXX"
GEMINI_API_KEY = "XXXXXXXXX"

In [4]:
# Consultar clima en API
city = "Medellin"
weather_response = requests.get(
    "https://api.openweathermap.org/data/2.5/weather",
    params={"q": city, "units": "metric", "appid": OPENWEATHER_API_KEY}
)

In [5]:
weather_data = weather_response.json()
print(weather_data)

{'coord': {'lon': -75.5636, 'lat': 6.2518}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 23.7, 'feels_like': 23.74, 'temp_min': 23.15, 'temp_max': 24.87, 'pressure': 1016, 'humidity': 62, 'sea_level': 1016, 'grnd_level': 808}, 'visibility': 10000, 'wind': {'speed': 1.54, 'deg': 0}, 'clouds': {'all': 20}, 'dt': 1750004346, 'sys': {'type': 2, 'id': 2099918, 'country': 'CO', 'sunrise': 1749984502, 'sunset': 1750029439}, 'timezone': -18000, 'id': 3674962, 'name': 'Medellín', 'cod': 200}


In [6]:
# Obtener temperatura y clima actual
temperature = weather_data["main"]["temp"]
description = weather_data["weather"][0]["description"]

In [7]:
# Crea el prompt para la API de Gemini
prompt = f"""
El clima actual en {city} es:
Temperatura: {temperature}°C
Condición: {description}

¿Qué ropa debería usar hoy?
"""

In [8]:
# Usar LangChain con Gemini
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7, google_api_key=GEMINI_API_KEY)
response = llm([HumanMessage(content=prompt)])

<ipython-input-8-3460815109>:3: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt)])


In [9]:
print("🤖 Respuesta de Gemini:\n")
print(response.content)

🤖 Respuesta de Gemini:

Con una temperatura de 23.7°C y cielo con pocas nubes en Medellín, el clima es bastante agradable y cálido. Aquí te sugiero algunas opciones de ropa:

*   **Parte superior:** Una camiseta de manga corta, blusa ligera o una camisa de lino serían ideales.
*   **Parte inferior:** Pantalones cortos, una falda, un vestido ligero o pantalones de tela delgada son buenas opciones.
*   **Calzado:** Sandalias, tenis o zapatos cómodos para caminar.
*   **Extras:**
    *   Lleva contigo un suéter ligero o una chaqueta delgada por si la temperatura baja un poco más tarde en el día o en lugares con aire acondicionado.
    *   Gafas de sol y protector solar son recomendables para protegerte del sol.
    *   Un paraguas pequeño podría ser útil, ya que en Medellín el clima puede ser impredecible.

En general, lo mejor es optar por ropa cómoda y transpirable que te permita disfrutar del clima cálido.


In [10]:
# Crea el prompt para la API de Gemini
prompt = f"""
El clima actual en {city} es:
Temperatura: {temperature}°C
Condición: {description}

Dame sugerencias de planes que podría hacer hoy en la ciudad
"""

In [11]:
# Usar LangChain con Gemini
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7, google_api_key=GEMINI_API_KEY)
response = llm([HumanMessage(content=prompt)])

In [12]:
print("🤖 Respuesta de Gemini:\n")
print(response.content)

🤖 Respuesta de Gemini:

¡Excelente! Con ese clima agradable de 23.7°C y pocas nubes, Medellín ofrece muchas opciones para disfrutar el día. Aquí te dejo algunas sugerencias, considerando que tienes un clima agradable:

**Actividades al aire libre:**

*   **Parque Arví:** Toma el Metrocable hasta la estación Santo Domingo y luego otro cable hasta el parque. Disfruta de caminatas ecológicas, picnics, mercados artesanales y la naturaleza. ¡Es un escape perfecto del bullicio de la ciudad!
*   **Jardín Botánico Joaquín Antonio Uribe:** Un oasis de tranquilidad en medio de la ciudad. Explora la diversidad de flora colombiana, relájate en sus jardines y visita el Orquideorama.
*   **Caminata por El Poblado:** Explora el barrio El Poblado, conocido por sus cafés, restaurantes, tiendas de diseño y vida nocturna. Puedes caminar por la Milla de Oro, visitar el Parque Lleras y disfrutar del ambiente cosmopolita.
*   **Paseo en bicicleta:** Alquila una bicicleta y recorre la ciudad. Medellín tiene 

# **LangChain usando un PDF**

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# Cargar y dividir el documento
loader = PyPDFLoader("/content/drive/MyDrive/Cursos/Python 2024/Los Gigantes que Dominaron la Tierra.pdf")
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
documents = splitter.split_documents(pages)

In [16]:
# Vectorizar con Gemini Embeddings y FAISS
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)
vectorstore = FAISS.from_documents(documents, embeddings)

In [17]:
# Usar QA Chain
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3, google_api_key=GEMINI_API_KEY)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

In [18]:
# Realizar pregunta
question = "¿Cuál es el propósito principal de este documento?"
response = qa_chain({"query": question})

<ipython-input-18-650050549>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": question})


In [19]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: ¿Cuál es el propósito principal de este documento?
🤖 Respuesta: El propósito principal de este documento es describir el legado de los dinosaurios desde diferentes perspectivas: científica, cultural y económica.


In [20]:
# Realizar pregunta
question = "Resume en cinco palabras este documento"
response = qa_chain({"query": question})

In [21]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: Resume en cinco palabras este documento
🤖 Respuesta: Dinosaurios, extinción, impacto, diversidad, adaptación.


In [22]:
# Realizar pregunta
question = "Hablame del tiranosaurio rex"
response = qa_chain({"query": question})

In [23]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: Hablame del tiranosaurio rex
🤖 Respuesta: El Tiranosaurio Rex fue un depredador ápice del Cretácico superior.


In [24]:
# Realizar pregunta
question = "¿Qué pasó en el periodo Cretácico?"
response = qa_chain({"query": question})

In [25]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: ¿Qué pasó en el periodo Cretácico?
🤖 Respuesta: En el período Cretácico, un asteroide gigante impactó la Tierra, lo que generó inviernos de impacto, incendios forestales a gran escala y tsunamis gigantescos. Los dinosaurios dominaron los ecosistemas terrestres debido a su capacidad de adaptación, diversidad y eficiente sistema respiratorio. Algunos de los dinosaurios más famosos de este período incluyen el Tiranosaurio Rex, el Velociraptor, el Braquiosaurio y el Triceratops.


In [26]:
# Realizar pregunta
question = "¿Qué relación hay entre los dinosaurios y las aves?"
response = qa_chain({"query": question})

In [27]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: ¿Qué relación hay entre los dinosaurios y las aves?
🤖 Respuesta: El descubrimiento de fósiles como el Archaeopteryx y el Microraptor ha revelado una transición gradual entre los dinosaurios terópodos y las aves modernas. El estudio de los embriones de dinosaurios y aves ha revelado sorprendentes similitudes en el desarrollo embrionario, lo que refuerza la teoría de que las aves evolucionaron a partir de los dinosaurios.


In [28]:
# Realizar pregunta
question = "¿Qué piensas de los dinosaurios y la cultura popular?"
response = qa_chain({"query": question})

In [29]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: ¿Qué piensas de los dinosaurios y la cultura popular?
🤖 Respuesta: Los dinosaurios han trascendido los libros de texto y los museos, convirtiéndose en iconos culturales. Su imagen ha sido moldeada y reinterpretada a lo largo de décadas, dejando una huella imborrable en el cine, la literatura, los videojuegos y otros medios de expresión.


In [30]:
# Cargar y dividir el documento
loader = PyPDFLoader("/content/drive/MyDrive/Cursos/Python 2024/El Renacimiento.pdf")
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
documents = splitter.split_documents(pages)

In [31]:
# Vectorizar con Gemini Embeddings y FAISS
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)
vectorstore = FAISS.from_documents(documents, embeddings)

In [32]:
# Usar QA Chain
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3, google_api_key=GEMINI_API_KEY)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

In [33]:
# Realizar pregunta
question = "¿Cuál es el propósito principal de este documento?"
response = qa_chain({"query": question})

In [34]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: ¿Cuál es el propósito principal de este documento?
🤖 Respuesta: El propósito principal del documento es describir el Renacimiento como un movimiento cultural y artístico que marcó una transición entre la Edad Media y la modernidad en Europa, destacando sus características, significado e impacto en diversos campos del conocimiento y la cultura.


In [35]:
# Realizar pregunta
question = "Resume en cinco palabras este documento"
response = qa_chain({"query": question})

In [36]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: Resume en cinco palabras este documento
🤖 Respuesta: Renacimiento: arte, cultura, conocimiento, humanismo.


In [37]:
# Realizar pregunta
question = "¿Cómo se relaciona la filosofía con el renacimiento?"
response = qa_chain({"query": question})

In [38]:
print("❓ Pregunta:", question)
print("🤖 Respuesta:", response["result"])

❓ Pregunta: ¿Cómo se relaciona la filosofía con el renacimiento?
🤖 Respuesta: El Renacimiento tuvo como corriente filosófica central el humanismo. Los humanistas se centraron en el estudio de las humanidades (filosofía, historia, literatura, etc.) y en la dignidad y potencial del ser humano. El humanismo renacentista influyó en el desarrollo de los derechos humanos y la democracia.


# **LangChain usando un audio**

In [39]:
# Configurar modelo
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 60.0MiB/s]


In [40]:
result = model.transcribe("/content/drive/MyDrive/Cursos/Python 2024/Transcribir audio/Audio_2.mp3")
transcribed_text = result["text"]

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [41]:
print(transcribed_text)

 Bonjour, aujourd'hui je vais vous parler de Verneuys sur scène. C'est une ville en France à 30 km de Paris. Verneuys est écrit V-E-R-N-E-U-I-L. Cette ville s'appelle Verneuys sur scène car elle est traversée par le fleuve La Saine. Il existe un autre Verneuys qui s'appelle Verneuys sur Avres car elle est traversée par le Avres. Nous, à Verneuys sur scène, nous sommes à peu près 15 000 habitants. C'est une ville de petite ville. J'aime le côté culturel dans cette ville. Il y a une salle de cinéma et une salle de spectacles, une école de danse et une école de musique. Au point de vue sportif, nous sommes bien équipés puisqu'il y a une piscine, un terrain de basket, un centre-éceste, un stade de foot, des terrains de tennis, un boulot drôme, un terrain pour jouer à la pétanque. Il y a aussi des pistes d'atletismes. Au point de vue nature, c'est vraiment une ville très agréable à vivre, puisque nous sommes juste à côté d'une énorme forêt dans laquelle c'est agréable de se promener, de cou

In [42]:
# Usar Gemini con LangChain
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3, google_api_key=GEMINI_API_KEY)

In [43]:
# Crear el prompt con la transcripción como contexto
prompt_base = f"""
Este es el texto transcrito de un audio:
{transcribed_text}

Responde:
"""

In [44]:
prompt = prompt_base + "¿De qué trata esta conversación?"
response = llm([HumanMessage(content=prompt)])

In [45]:
print("🤖 Gemini responde:")
print(response.content)

🤖 Gemini responde:
La conversación trata sobre la ciudad de Verneuil-sur-Seine, en Francia. La persona que habla describe la ciudad, su ubicación, su nombre, el número de habitantes, las instalaciones culturales y deportivas, la naturaleza que la rodea, las opciones educativas y su proximidad a París y la costa de Normandía. También invita a la gente a visitar el sitio web y el grupo de Facebook de la ciudad para obtener más información.


In [46]:
prompt = prompt_base + "Hablame de sus instalaciones culturales"
response = llm([HumanMessage(content=prompt)])

In [47]:
print("🤖 Gemini responde:")
print(response.content)

🤖 Gemini responde:
Según el audio, Verneuys-sur-Seine tiene una rica oferta cultural:

*   **Cine:** Hay una sala de cine.
*   **Espectáculos:** Hay una sala de espectáculos.
*   **Danza:** Hay una escuela de danza.
*   **Música:** Hay una escuela de música.


In [48]:
prompt = prompt_base + "¿Qué se habla acerca de París?"
response = llm([HumanMessage(content=prompt)])

In [49]:
print("🤖 Gemini responde:")
print(response.content)

🤖 Gemini responde:
Se menciona que Verneuil-sur-Seine está a solo 30 minutos de París y que, gracias a la autopista, se puede llegar a la costa de Normandía en dos horas.


In [50]:
prompt = prompt_base + "¿Qué me puedes decir de su entorno natural?"
response = llm([HumanMessage(content=prompt)])

In [51]:
print("🤖 Gemini responde:")
print(response.content)

🤖 Gemini responde:
Según el audio, el entorno natural de Verneuil-sur-Seine es muy agradable. Destaca lo siguiente:

*   **Proximidad a un gran bosque:** Ideal para pasear, correr y recoger setas.
*   **Campos:** Hay muchos campos de maíz, colza y trigo.
*   **El río Sena:** Atraviesa la ciudad, lo que hace agradable pasear por sus orillas.
*   **Ubicación:** Está en el campo, pero cerca de París y de la costa de Normandía.


In [52]:
prompt = prompt_base + "Hablame acerca de la calidad de vida"
response = llm([HumanMessage(content=prompt)])

In [53]:
print("🤖 Gemini responde:")
print(response.content)

🤖 Gemini responde:
Según el audio, la calidad de vida en Verneuil-sur-Seine parece ser bastante buena, destacando varios aspectos:

*   **Ubicación:** Está cerca de París (a 30 km), lo que permite acceder a las ventajas de una gran ciudad, pero manteniendo un ambiente más tranquilo y campestre. Además, está a dos horas de la costa de Normandía, lo que facilita escapadas de fin de semana.
*   **Naturaleza:** La ciudad está rodeada de naturaleza, con una gran forêt para pasear, correr y recolectar setas, así como campos de maíz, colza y trigo. El río Sena también atraviesa la ciudad, ofreciendo agradables paseos a orillas del río.
*   **Equipamiento:** La ciudad cuenta con buenas instalaciones deportivas (piscina, campos de baloncesto, centro ecuestre, estadio de fútbol, pistas de tenis, etc.) y culturales (cine, sala de espectáculos, escuela de danza y música).
*   **Educación:** Ofrece una amplia gama de opciones educativas para todas las edades, desde guarderías hasta institutos, tant

In [54]:
prompt = prompt_base + "Complementa a lo que menciona el audio, con tu conocimiento acerca de que actividades es común realizar en la ciudad"
response = llm([HumanMessage(content=prompt)])

In [55]:
print("🤖 Gemini responde:")
print(response.content)

🤖 Gemini responde:
De acuerdo, complementando la descripción de Verneuil-sur-Seine que se da en el audio, y basándome en lo que es común encontrar en ciudades francesas similares y en información general sobre la región, puedo añadir lo siguiente sobre las actividades que se pueden realizar allí:

**Más allá de lo mencionado en el audio, en Verneuil-sur-Seine probablemente se puedan disfrutar de:**

*   **Mercados locales:** Es muy probable que Verneuil-sur-Seine tenga un mercado semanal o varios mercados a la semana donde se puedan comprar productos frescos de la región, quesos, carnes, panes y otros productos artesanales. Estos mercados son un punto de encuentro social importante.
*   **Festivales y eventos culturales:** Además de los eventos que publique el ayuntamiento en Facebook, Verneuil-sur-Seine podría tener festivales de música, ferias de artesanía, celebraciones de fiestas nacionales (como el 14 de julio) o eventos relacionados con la gastronomía local.
*   **Actividades al 

In [56]:
prompt = f"""
Analiza el siguiente texto transcrito de una conversación.
Devuélveme un análisis emocional con:

1. Emoción predominante (alegría, tristeza, enojo, miedo, sorpresa, neutralidad)
2. Sentimientos asociados (por ejemplo: frustración, esperanza, ansiedad)
3. Justificación de por qué llegaste a esa conclusión

Texto:
{transcribed_text}
"""

In [57]:
response = llm([HumanMessage(content=prompt)])

In [58]:
print("🔍 Análisis de emociones:")
print(response.content)

🔍 Análisis de emociones:
Aquí está el análisis emocional del texto:

1.  **Emoción predominante:** Neutralidad con toques de alegría.

2.  **Sentimientos asociados:**

    *   **Orgullo:** La persona parece orgullosa de su ciudad y de lo que ofrece.
    *   **Aprecio:** Se nota un aprecio por la naturaleza, la cultura y las facilidades que ofrece Verneuys sur scène.
    *   **Entusiasmo moderado:** Hay un entusiasmo contenido al describir los aspectos positivos de la ciudad.

3.  **Justificación:**

    *   El tono general del texto es informativo y descriptivo, lo que indica neutralidad. La persona está presentando hechos sobre la ciudad.
    *   Sin embargo, hay indicios de alegría y orgullo en frases como:
        *   "J'aime le côté culturel dans cette ville." (Me gusta el lado cultural de esta ciudad).
        *   "Au point de vue nature, c'est vraiment une ville très agréable à vivre" (Desde el punto de vista de la naturaleza, es realmente una ciudad muy agradable para vivir).
  

In [59]:
prompt = f"""
Analiza el siguiente texto transcrito de una conversación.
Resume los momentos de mayor carga emocional y explica qué emociones estaban involucradas

Texto:
{transcribed_text}
"""

In [60]:
response = llm([HumanMessage(content=prompt)])

In [61]:
print("🔍 Análisis de emociones:")
print(response.content)

🔍 Análisis de emociones:
El texto transcrito es una descripción de la ciudad de Verneuil-sur-Seine.  Aunque el tono general es informativo y descriptivo, podemos identificar algunos momentos con una ligera carga emocional, principalmente positiva.

**Momentos de mayor carga emocional:**

*   **"J'aime le côté culturel dans cette ville." (Me gusta el lado cultural de esta ciudad.)** Aquí, la emoción involucrada es el **gusto** o el **aprecio**. La persona expresa una preferencia personal por los aspectos culturales de la ciudad.
*   **"Au point de vue nature, c'est vraiment une ville très agréable à vivre..." (Desde el punto de vista de la naturaleza, es realmente una ciudad muy agradable para vivir...)**  La emoción principal aquí es el **entusiasmo** o la **satisfacción**. La persona está destacando las cualidades positivas de la ciudad en términos de su entorno natural. La frase "très agréable à vivre" (muy agradable para vivir) sugiere una fuerte conexión emocional positiva con el l

# **LangChain con una base de datos SQL Server**

In [78]:
# Datos de conexión
host = "database-1.clomociw036x.us-east-2.rds.amazonaws.com"
database = "Northwind"
username = "XXXXX"
password = "XXXXXX"
port = 1433

In [11]:
# Crear engine con pytds
engine = create_engine(f"mssql+pytds://{username}:{password}@{host}:{port}/{database}")

In [12]:
# Crear objeto LangChain para la base de datos
db = SQLDatabase(engine)

In [21]:
# Usar LangChain con Gemini AI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3, google_api_key=GEMINI_API_KEY)

In [22]:
# Esquema de la base de datos
schema = """
Tables:
Products(ProductID, ProductName, SupplierID, CategoryID, QuantityPerUnit, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued)
Suppliers(SupplierID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone)
Categories(CategoryID, CategoryName, Description)

Relaciones:
Products.SupplierID → Suppliers.SupplierID
Products.CategoryID → Categories.CategoryID
"""

In [60]:
# Prompt para que Gemini genere SQL
prompt = PromptTemplate(
    input_variables=["question", "schema"],
    template="""
Eres un experto en bases de datos. Genera únicamente una consulta SQL válida para SQL Server que responda la siguiente pregunta, usando el esquema dado.

No incluyas explicaciones, encabezados ni etiquetas como "sql:" o "Aquí está la consulta".
Solo responde con la consulta SQL.

Esquema:
{schema}

Pregunta:
{question}
""".strip()
)

In [38]:
# Pasar prompt
chain = LLMChain(llm=llm, prompt=prompt)

In [47]:
# Hacer pregunta
question = "¿Cuáles son los nombres de productos del proveedor 'Exotic Liquids' y su categoría?"
sql = chain.run({"question": question, "schema": schema})

In [48]:
sql = sql.replace("sql", "").replace("```", "").strip()

In [49]:
print("🤖 Gemini responde:")
print(sql)

🤖 Gemini responde:
SELECT p.ProductName, c.CategoryName
FROM Products p
JOIN Suppliers s ON p.SupplierID = s.SupplierID
JOIN Categories c ON p.CategoryID = c.CategoryID
WHERE s.CompanyName = 'Exotic Liquids';


In [51]:
# Ejecutar consulta
response = db.run(sql)
print("Resultado:\n", response)

Resultado:
 [('Chai', 'Beverages'), ('Chang', 'Beverages'), ('Aniseed Syrup', 'Condiments')]


In [52]:
# Hacer pregunta
question = "¿Cuántos productos hay por categoría?"
sql = chain.run({"question": question, "schema": schema})

In [53]:
sql = sql.replace("sql", "").replace("```", "").strip()

In [54]:
print("🤖 Gemini responde:")
print(sql)

🤖 Gemini responde:
SELECT c.CategoryName, COUNT(p.ProductID) AS NumberOfProducts
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
GROUP BY c.CategoryName;


In [55]:
# Ejecutar consulta
response = db.run(sql)
print("Resultado:\n", response)

Resultado:
 [('Beverages', 12), ('Condiments', 12), ('Confections', 13), ('Dairy Products', 10), ('Grains/Cereals', 7), ('Meat/Poultry', 6), ('Produce', 5), ('Seafood', 12)]


In [61]:
# Hacer pregunta
question = "¿Cuál es el producto más caro y quién lo provee?"
sql = chain.run({"question": question, "schema": schema})

In [62]:
sql = sql.replace("sql", "").replace("```", "").strip()

In [63]:
print("🤖 Gemini responde:")
print(sql)

🤖 Gemini responde:
SELECT TOP 1 p.ProductName, s.CompanyName
FROM Products p
JOIN Suppliers s ON p.SupplierID = s.SupplierID
ORDER BY p.UnitPrice DESC;


In [64]:
# Ejecutar consulta
response = db.run(sql)
print("Resultado:\n", response)

Resultado:
 [('Côte de Blaye', 'Aux joyeux ecclésiastiques')]


In [69]:
# Hacer pregunta
question = "¿Qué proveedores venden productos con precio menor a 10?"
sql = chain.run({"question": question, "schema": schema})

In [70]:
sql = sql.replace("sql", "").replace("```", "").strip()

In [71]:
print("🤖 Gemini responde:")
print(sql)

🤖 Gemini responde:
SELECT DISTINCT s.CompanyName
FROM Suppliers s
JOIN Products p ON s.SupplierID = p.SupplierID
WHERE p.UnitPrice < 10;


In [72]:
# Ejecutar consulta
response = db.run(sql)
print("Resultado:\n", response)

Resultado:
 [("G'day, Mate",), ('Lyngbysild',), ('Ma Maison',), ("Mayumi's",), ('New England Seafood Cannery',), ('Norske Meierier',), ('PB Knäckebröd AB',), ('Plutzer Lebensmittelgroßmärkte AG',), ('Refrescos Americanas LTDA',), ('Specialty Biscuits, Ltd.',), ('Zaanse Snoepfabriek',)]


In [73]:
# Hacer pregunta
question = "¿Cuál es el precio promedio por categoría?"
sql = chain.run({"question": question, "schema": schema})

In [74]:
sql = sql.replace("sql", "").replace("```", "").strip()

In [75]:
print("🤖 Gemini responde:")
print(sql)

🤖 Gemini responde:
SELECT c.CategoryName, AVG(p.UnitPrice) AS AveragePrice
FROM Products p
JOIN Categories c ON p.CategoryID = c.CategoryID
GROUP BY c.CategoryName;


In [76]:
# Ejecutar consulta
response = db.run(sql)
print("Resultado:\n", response)

Resultado:
 [('Beverages', Decimal('37.9791')), ('Condiments', Decimal('23.0625')), ('Confections', Decimal('25.16')), ('Dairy Products', Decimal('28.73')), ('Grains/Cereals', Decimal('20.25')), ('Meat/Poultry', Decimal('54.0066')), ('Produce', Decimal('32.37')), ('Seafood', Decimal('20.6825'))]
